<a href="https://colab.research.google.com/github/2021aim1014/Master-Thesis-Project/blob/main/Our_results_on_political_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# %pip install poutyne          # to install the Poutyne library
# %pip install wget             # to install the wget library in order to download data
# %pip install opencv-python 

In [ ]:
import os
import matplotlib.pyplot as plt
import numpy as np
import natsort
import pandas as pd 
import scipy
import shutil
import cv2
import math
import random
import torch
from torch import nn, optim
from torchvision import datasets, transforms
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
# import wget
import zipfile

from tensorflow.keras.utils  import plot_model
from keras.models import Model
from keras.layers import Input, Dense, Flatten, ZeroPadding2D
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

import tensorflow as tf

from tensorflow.keras import datasets, layers, models

from numpy import array
from numpy import argmax

Load image dataset

In [ ]:
def load_dataset(image_folder, training_df):                                                              # New image dimensions
  new_w,new_h = 28,28                                                               # New image dimensions

  trainingImages = []
  trainingLabels = []
  train_set_size, _ = training_df.shape
  i=0
  for i in range(train_set_size):
    filePath = image_folder+training_df['image_id'][i]
    frame    = cv2.imread(filePath)/255
    resize   = cv2.resize(frame,(new_w,new_h))                                      # Normalizing
    trainingImages.append(resize)
    trainingLabels.append(training_df['Smiling'][i])
    if(i%10000 == 0):
      print(i,"images loaded")


  print("Training images loaded: ",i+1,"images loaded\n------------------------------------------------------------------------------------\n")
  trainingImages   = np.asarray(trainingImages)
  return (trainingImages, trainingLabels)

validation_df

### Normal Convolutional Neural Network

In [ ]:
def one_hot_encoder(trainingLabels):
  trainingLabels   = np.asarray(trainingLabels)
  
  tr_label_encoder = LabelEncoder()
  tr_integer_encoded = tr_label_encoder.fit_transform(trainingLabels)

  tr_onehot_encoder = OneHotEncoder(sparse=False)
  tr_integer_encoded = tr_integer_encoded.reshape(len(tr_integer_encoded), 1)
  tr_onehot_encoded = tr_onehot_encoder.fit_transform(tr_integer_encoded)
  tr_onehot_encoded = tr_onehot_encoded.astype(int)
  return tr_onehot_encoded

In [ ]:
import os
from google.colab import drive
drive.mount('/content/drive')
path = '/content/drive/MyDrive/MTP_dataset/G500B500/'

Mounted at /content/drive


In [ ]:
df = pd.read_csv('/content/drive/MyDrive/MTP_dataset/label_result.csv')

In [ ]:
df.replace(to_replace='green', value=0, inplace=True)
df.replace(to_replace='blue', value=1, inplace=True)

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size=0.3)

In [ ]:
true_labels = np.asarray(train['publisher'])
human_labels = np.asarray(train['human'])
imagePath = np.asarray(train['file'])

In [ ]:
# Train Image Loader
new_w,new_h = 28,28                                                               # New image dimensions

trainingImages = []
trainingLabels = []
human_training_labels = []
train_set_size, _ = train.shape
i=0
for i in range(train_set_size):
  if(i%100 == 0): print(i)
  if(true_labels[i] == 1):
    filePath = path+'Blue500/'+imagePath[i]
  else:
    filePath = path+'Green500/'+imagePath[i]
  frame    = cv2.imread(filePath)/255
  resize   = cv2.resize(frame,(new_w,new_h))                                      # Normalizing
  trainingImages.append(resize)
  trainingLabels.append(true_labels[i])
  human_training_labels.append(human_labels[i])

In [ ]:
trainingImages   = np.asarray(trainingImages)
trainingLabels   = np.asarray(trainingLabels)
human_training_labels   = np.asarray(human_training_labels)

In [ ]:
trainingImages.shape

(910, 28, 28, 3)

In [ ]:
# Test Image Loader
true_labels = np.asarray(test['publisher'])
human_labels = np.asarray(test['human'])
imagePath = np.asarray(test['file'])
new_w,new_h = 28,28                                                               # New image dimensions

testImages = []
testLabels = []
human_test_labels = []

size, _ = test.shape
i=0
for i in range(size):
  if(i%100 == 0):
     print(i)
  if(true_labels[i] == 1):
    filePath = path+'Blue500/'+imagePath[i]
  else:
    filePath = path+'Green500/'+imagePath[i]
  frame    = cv2.imread(filePath)/255
  resize   = cv2.resize(frame,(new_w,new_h))                                      # Normalizing
  testImages.append(resize)
  testLabels.append(true_labels[i])
  human_test_labels.append(human_labels[i])



0
100
200
300


In [ ]:
testImages   = np.asarray(testImages)
testLabels   = np.asarray(testLabels)
human_test_labels   = np.asarray(human_test_labels)

In [ ]:
# One hot encoding
tr_onehot_encoded = one_hot_encoder(trainingLabels)
test_onehot_encoded = one_hot_encoder(testLabels)

In [ ]:
# ===============================================================================
#   LeNet
# ===============================================================================

model = Sequential()
model.add(Conv2D(input_shape=(28,28,3),filters=6,kernel_size=(5,5),strides=(1,1),padding="same", activation="tanh"))
model.add(layers.AveragePooling2D(pool_size=(2,2),strides=(1,1),padding="valid"))
model.add(Conv2D(filters=16, kernel_size=(5,5),strides=(1,1), padding="valid", activation="tanh"))
model.add(layers.AveragePooling2D(pool_size=(2,2),strides=(2,2),padding="valid"))
model.add(Conv2D(filters=120, kernel_size=(5,5),strides=(1,1), padding="valid", activation="tanh"))
model.add(Flatten())
model.add(Dense(units=84,activation="tanh"))
model.add(Dense(units=2,activation="softmax"))

In [ ]:
# -------------------------------------------------------------------------------------------------------------------------------------
# Compiling the model
# -------------------------------------------------------------------------------------------------------------------------------------

from tensorflow.keras.optimizers import SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax, Nadam

model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy','mse','mae'])

# -------------------------------------------------------------------------------------------------------------------------------------
# Training the model on Training data set
# -------------------------------------------------------------------------------------------------------------------------------------

my_history = model.fit(trainingImages, tr_onehot_encoded, epochs=1, batch_size=4,verbose=1, validation_split=0.1)

y_pred = model.predict(testImages)
Y_pred = np.argmax(y_pred, axis=1)
N = Y_pred.shape[0]
y = Y_pred.reshape((N,1))
keras.metrics.categorical_accuracy(test_onehot_encoded, y)

score = model.evaluate(testImages, test_onehot_encoded, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

13/13 [==============================] - 0s 4ms/step - loss: 0.6062 - accuracy: 0.7136 - mse: 0.2067 - mae: 0.4278
accuracy: 71.36%


# Testing

### Test on test dataset

In [ ]:
y_pred = model.predict(testImages)
Y_pred = np.argmax(y_pred, axis=1)
N = Y_pred.shape[0]
y = Y_pred.reshape((N,1))
keras.metrics.categorical_accuracy(test_onehot_encoded, y)

score = model.evaluate(testImages, test_onehot_encoded, verbose=1)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

print("_________________________________________________________________________________________________________________________________________\n")

13/13 [==============================] - 0s 4ms/step - loss: 0.6062 - accuracy: 0.7136 - mse: 0.2067 - mae: 0.4278
accuracy: 71.36%
_________________________________________________________________________________________________________________________________________



In [ ]:
human_test_labels = np.asarray(human_test_labels)
testLabels = np.asarray(testLabels)

print(np.mean(human_test_labels == testLabels))

0.959079283887468


In [ ]:
result = {
    'Missclassification Cost':[],
    'Human Cost':[],
    'Combined': [],
    'Human correct': [],
    'Human refered': [],
    'Human accuracy': [],
    'Model correct': [],
    'Model refered': [],
    'Model accuracy': [],
    'Loss': []
}

In [ ]:
for human_cost in range(10):
    for miss_cost in range(human_cost+1, 10):
        correct = 0
        total = testLabels.shape[0]
        combined_accuracy = 0
        human_correct = 0
        human_refered = 0
        human_accuracy = 0
        model_correct = 0
        model_refered = 0
        model_accuracy = 0
        loss = 0
        for i in range(total):
            P_X = miss_cost * (1 - y_pred[i][Y_pred[i]])
            if(human_cost <= P_X):
                human_refered += 1
                if(testLabels[i] == human_test_labels[i]):
                    human_correct += 1
                    correct += 1
            else:
                model_refered += 1
                if(Y_pred[i] == testLabels[i]):
                    model_correct += 1
                    correct += 1

        combined_accuracy = 100 * correct / total
        human_accuracy = 100 * human_correct / (human_refered + 0.000001)
        model_accuracy = 100 * model_correct / (model_refered + 0.000001)
        loss = miss_cost * (model_refered - model_correct) + human_cost * human_refered

        result['Missclassification Cost'].append(miss_cost)
        result['Human Cost'].append(human_cost)
        result['Combined'].append(combined_accuracy)
        result['Human correct'].append(human_correct)
        result['Human refered'].append(human_refered)
        result['Human accuracy'].append(human_accuracy)
        result['Model correct'].append(model_correct)
        result['Model refered'].append(model_refered)
        result['Model accuracy'].append(model_accuracy)
        result['Loss'].append(loss)


In [ ]:
pd.DataFrame.from_dict(result)[10:29]

,Missclassification Cost,Human Cost,Combined,Human correct,Human refered,Human accuracy,Model correct,Model refered,Model accuracy,Loss
10,3,1,90.537084,255,270,94.444444,99,121,81.818181,336
11,4,1,93.094629,319,335,95.223880,45,56,80.357141,379
12,5,1,93.861893,343,359,95.543175,24,32,74.999998,399
13,6,1,94.629156,353,369,95.663956,17,22,77.272724,399
14,7,1,95.140665,361,377,95.755968,11,14,78.571423,398
15,8,1,95.396419,365,381,95.800525,8,10,79.999992,397
16,9,1,95.396419,365,381,95.800525,8,10,79.999992,399
17,3,2,71.355499,0,0,0.000000,279,391,71.355499,336
18,4,2,71.355499,0,0,0.000000,279,391,71.355499,448
19,5,2,85.166240,164,172,95.348837,169,219,77.168949,594


In [ ]:
df.to_csv('file1.csv')